# 14_Ensemble_FT_MLP_v2 요약

- 모델: FT-Transformer + MLP 앙상블 (SEEDS=[42,2024,7777])
- 피처: new3 버전(원문항 + 파생 혼합)
- 학습/평가: KFold 5, seed 앙상블
- 제출파일: submission_14_Ensemble_FT_MLP_v2.csv


In [ ]:
"""
🎯 투표 예측 최종 모델 v7
==========================
검증된 18개 피처 (Permutation Importance 기반)
- Target Encoding이 핵심 (84% 중요도)
- MLP + FT-Transformer 앙상블
- 3 Seeds
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# CONFIG
# ============================================================
SEEDS = [42, 2024, 7777]
N_FOLDS = 5
BATCH_SIZE = 512
EPOCHS = 60
PATIENCE = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🖥️ Device: {DEVICE}")
print(f"🌱 Seeds: {SEEDS}")
print(f"📊 Total: {len(SEEDS)} seeds × 2 models × {N_FOLDS} folds = {len(SEEDS)*2*N_FOLDS} models")


def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


# ============================================================
# 데이터 로딩 & 전처리
# ============================================================
print("\n📂 데이터 로딩...")
train_raw = pd.read_csv("../../data/raw/train.csv")
test_raw = pd.read_csv("../../data/raw/test_x.csv")
train_raw["voted_bin"] = (train_raw["voted"] == 2).astype(int)
print(f"Train: {train_raw.shape}, Test: {test_raw.shape}")


def preprocess(df):
    df = df.copy()
    for col in ['education', 'married', 'urban', 'engnat']:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
    for col in [f"tp{i:02d}" for i in range(1, 11)]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
    return df


def build_base_features(df):
    df = df.copy()
    
    # age_ord
    age_map = {"10s": 1, "20s": 2, "30s": 3, "40s": 4, "50s": 5, "60s": 6, "+70s": 7}
    df["age_ord"] = df["age_group"].map(age_map)
    
    # Interaction
    df['age_edu'] = df['age_ord'] * df['education']
    df['is_teenager'] = (df['age_ord'] == 1).astype(int)
    df['teen_low_edu'] = df['is_teenager'] * (df['education'] <= 2).astype(float)
    
    # WR sum
    wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
    df['wr_sum'] = df[wr_cols].sum(axis=1)
    
    # 복합 카테고리 (TE용)
    df['age_edu_cat'] = df['age_group'].astype(str) + '_' + df['education'].astype(str)
    df['age_edu_mar_cat'] = df['age_group'].astype(str) + '_' + df['education'].astype(str) + '_' + df['married'].astype(str)
    
    return df


# ============================================================
# Target Encoding (CV leak 방지)
# ============================================================
def target_encode(train_df, val_df, test_df, col, target_col, smoothing=10):
    global_mean = train_df[target_col].mean()
    agg = train_df.groupby(col)[target_col].agg(['mean', 'count'])
    agg['te'] = (agg['count'] * agg['mean'] + smoothing * global_mean) / (agg['count'] + smoothing)
    te_map = agg['te'].to_dict()
    
    return (
        train_df[col].map(te_map).fillna(global_mean).values,
        val_df[col].map(te_map).fillna(global_mean).values,
        test_df[col].map(te_map).fillna(global_mean).values
    )


def create_te_features(train_df, val_df, test_df):
    te = {'train': {}, 'val': {}, 'test': {}}
    
    # 단일 TE
    for col, sm in [('age_group', 10), ('race', 10), ('religion', 10)]:
        tr, va, ts = target_encode(train_df, val_df, test_df, col, 'voted_bin', sm)
        te['train'][f'{col}_te'] = tr
        te['val'][f'{col}_te'] = va
        te['test'][f'{col}_te'] = ts
    
    # 복합 TE
    for col, sm in [('age_edu_cat', 5), ('age_edu_mar_cat', 3)]:
        tr, va, ts = target_encode(train_df, val_df, test_df, col, 'voted_bin', sm)
        te['train'][f'{col}_te'] = tr
        te['val'][f'{col}_te'] = va
        te['test'][f'{col}_te'] = ts
    
    return te


# ============================================================
# 피처 리스트 (검증된 18개)
# ============================================================
base_features = [
    # 원본 (5개)
    'age_ord', 'education', 'married', 'urban', 'engnat',
    # Interaction (4개)
    'age_edu', 'is_teenager', 'teen_low_edu', 'wr_sum',
    # TP 상위 (4개)
    'tp07', 'tp06', 'tp04', 'tp03',
]

te_features = [
    'age_group_te', 'race_te', 'religion_te',
    'age_edu_cat_te', 'age_edu_mar_cat_te',
]

print(f"\n📊 피처: base={len(base_features)}, TE={len(te_features)}, 총={len(base_features)+len(te_features)}")


# ============================================================
# Dataset & Models
# ============================================================
class TabDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.3):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.extend([
                nn.Linear(prev, h),
                nn.BatchNorm1d(h),
                nn.SiLU(),
                nn.Dropout(dropout)
            ])
            prev = h
        layers.append(nn.Linear(prev, 1))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)


class FTTransformer(nn.Module):
    def __init__(self, input_dim, d_token=64, n_layers=2, n_heads=4, dropout=0.2):
        super().__init__()
        self.embed = nn.Linear(1, d_token)
        self.cls = nn.Parameter(torch.randn(1, 1, d_token) * 0.02)
        
        encoder = nn.TransformerEncoderLayer(
            d_model=d_token, nhead=n_heads, dim_feedforward=d_token*4,
            dropout=dropout, activation='gelu', batch_first=True, norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder, num_layers=n_layers)
        self.head = nn.Sequential(
            nn.LayerNorm(d_token),
            nn.Linear(d_token, 1)
        )
        self.input_dim = input_dim
    
    def forward(self, x):
        # x: (batch, features) -> (batch, features, d_token)
        x = x.unsqueeze(-1)  # (batch, features, 1)
        x = self.embed(x)     # (batch, features, d_token)
        
        cls = self.cls.expand(x.size(0), -1, -1)
        x = torch.cat([cls, x], dim=1)
        x = self.transformer(x)
        return self.head(x[:, 0])


# ============================================================
# 학습 함수
# ============================================================
def train_model(model, train_loader, val_loader, val_y, device, lr=1e-3):
    model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10)
    
    best_auc = 0
    best_state = None
    no_improve = 0
    
    for epoch in range(EPOCHS):
        model.train()
        for batch in train_loader:
            x, y = batch[0].to(device), batch[1].to(device)
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
        scheduler.step()
        
        # Validation
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in val_loader:
                x = batch[0].to(device) if isinstance(batch, (list, tuple)) else batch.to(device)
                preds.append(torch.sigmoid(model(x)).cpu().numpy())
        
        preds = np.concatenate(preds).ravel()
        auc = roc_auc_score(val_y, preds)
        
        if auc > best_auc + 1e-5:
            best_auc = auc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
        
        if no_improve >= PATIENCE:
            break
    
    if best_state:
        model.load_state_dict(best_state)
    return model, best_auc


def predict(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in loader:
            x = batch[0].to(device) if isinstance(batch, (list, tuple)) else batch.to(device)
            preds.append(torch.sigmoid(model(x)).cpu().numpy())
    return np.concatenate(preds).ravel()


# ============================================================
# 메인
# ============================================================
def main():
    print("\n" + "="*60)
    print("🚀 학습 시작")
    print("="*60)
    
    train_clean = preprocess(train_raw)
    test_clean = preprocess(test_raw)
    
    train_fe = build_base_features(train_clean)
    test_fe = build_base_features(test_clean)
    
    # 결과 저장
    oof_mlp = np.zeros(len(train_fe))
    oof_ft = np.zeros(len(train_fe))
    test_mlp = np.zeros(len(test_fe))
    test_ft = np.zeros(len(test_fe))
    
    all_aucs = {'mlp': [], 'ft': []}
    
    for seed_idx, seed in enumerate(SEEDS):
        print(f"\n{'#'*60}")
        print(f"🌱 SEED {seed_idx+1}/{len(SEEDS)} (seed={seed})")
        print(f"{'#'*60}")
        
        set_seed(seed)
        
        seed_oof_mlp = np.zeros(len(train_fe))
        seed_oof_ft = np.zeros(len(train_fe))
        seed_test_mlp = np.zeros(len(test_fe))
        seed_test_ft = np.zeros(len(test_fe))
        
        skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=seed)
        
        for fold, (tr_idx, va_idx) in enumerate(skf.split(train_fe, train_fe["voted_bin"])):
            print(f"\n  📂 Fold {fold+1}/{N_FOLDS}")
            
            tr_df = train_fe.iloc[tr_idx].reset_index(drop=True)
            va_df = train_fe.iloc[va_idx].reset_index(drop=True)
            ts_df = test_fe.copy()
            
            # Target Encoding
            te = create_te_features(tr_df, va_df, ts_df)
            
            # 피처 합치기
            X_tr = tr_df[base_features].copy()
            X_va = va_df[base_features].copy()
            X_ts = ts_df[base_features].copy()
            
            for te_name in te_features:
                X_tr[te_name] = te['train'][te_name]
                X_va[te_name] = te['val'][te_name]
                X_ts[te_name] = te['test'][te_name]
            
            # NaN 처리
            for col in X_tr.columns:
                med = X_tr[col].median()
                X_tr[col] = X_tr[col].fillna(med)
                X_va[col] = X_va[col].fillna(med)
                X_ts[col] = X_ts[col].fillna(med)
            
            # 스케일링
            scaler = StandardScaler()
            X_tr_s = scaler.fit_transform(X_tr)
            X_va_s = scaler.transform(X_va)
            X_ts_s = scaler.transform(X_ts)
            
            y_tr = tr_df["voted_bin"].values
            y_va = va_df["voted_bin"].values
            
            # DataLoader
            tr_ds = TabDataset(X_tr_s, y_tr)
            va_ds = TabDataset(X_va_s, y_va)
            ts_ds = TabDataset(X_ts_s)
            
            tr_loader = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
            va_loader = DataLoader(va_ds, batch_size=BATCH_SIZE)
            ts_loader = DataLoader(ts_ds, batch_size=BATCH_SIZE)
            
            n_feat = X_tr_s.shape[1]
            
            # MLP
            mlp = MLP(n_feat, hidden_dims=[256, 128, 64], dropout=0.3)
            mlp, auc_mlp = train_model(mlp, tr_loader, va_loader, y_va, DEVICE, lr=1e-3)
            all_aucs['mlp'].append(auc_mlp)
            
            seed_oof_mlp[va_idx] = predict(mlp, va_loader, DEVICE)
            seed_test_mlp += predict(mlp, ts_loader, DEVICE) / N_FOLDS
            
            # FT-Transformer
            ft = FTTransformer(n_feat, d_token=64, n_layers=2, n_heads=4, dropout=0.2)
            ft, auc_ft = train_model(ft, tr_loader, va_loader, y_va, DEVICE, lr=5e-4)
            all_aucs['ft'].append(auc_ft)
            
            seed_oof_ft[va_idx] = predict(ft, va_loader, DEVICE)
            seed_test_ft += predict(ft, ts_loader, DEVICE) / N_FOLDS
            
            print(f"     MLP: {auc_mlp:.5f}, FT: {auc_ft:.5f}")
        
        # Seed 평균
        oof_mlp += seed_oof_mlp / len(SEEDS)
        oof_ft += seed_oof_ft / len(SEEDS)
        test_mlp += seed_test_mlp / len(SEEDS)
        test_ft += seed_test_ft / len(SEEDS)
    
    # ============================================
    # 최종 결과
    # ============================================
    print("\n" + "="*60)
    print("🎯 최종 결과")
    print("="*60)
    
    auc_mlp = roc_auc_score(train_fe["voted_bin"], oof_mlp)
    auc_ft = roc_auc_score(train_fe["voted_bin"], oof_ft)
    
    print(f"\n📊 MLP OOF: {auc_mlp:.5f} (mean: {np.mean(all_aucs['mlp']):.5f})")
    print(f"📊 FT OOF:  {auc_ft:.5f} (mean: {np.mean(all_aucs['ft']):.5f})")
    
    # 최적 가중치 탐색
    best_w, best_auc = 0.5, 0
    for w in np.arange(0.2, 0.8, 0.05):
        blend = w * oof_mlp + (1-w) * oof_ft
        auc = roc_auc_score(train_fe["voted_bin"], blend)
        if auc > best_auc:
            best_w, best_auc = w, auc
    
    print(f"\n🏆 최적 앙상블 (MLP:{best_w:.2f} + FT:{1-best_w:.2f}): {best_auc:.5f}")
    
    # 최종 예측
    final = best_w * test_mlp + (1-best_w) * test_ft
    
    # 저장
    sub = pd.DataFrame({
        "index": test_raw["index"] if "index" in test_raw.columns else range(len(test_raw)),
        "voted": final
    })
    sub.to_csv("submission_14_Ensemble_FT_MLP_v2_mlp.csv", index=False)
    
    # 개별도 저장
    pd.DataFrame({"index": sub["index"], "voted": test_mlp}).to_csv("submission_14_Ensemble_FT_MLP_v2_mlp.csv", index=False)
    pd.DataFrame({"index": sub["index"], "voted": test_ft}).to_csv("submission_14_Ensemble_FT_MLP_v2_mlp.csv", index=False)
    
    print(f"\n💾 저장: submission_.csv")
    print(f"   예측 범위: [{final.min():.4f}, {final.max():.4f}]")
    
    return best_auc


if __name__ == "__main__":
    main()



🖥️ Device: cpu
🌱 Seeds: [42, 2024, 7777]
📊 Total: 3 seeds × 2 models × 5 folds = 30 models

📂 데이터 로딩...
Train: (45532, 79), Test: (11383, 77)

📊 피처: base=13, TE=5, 총=18

🚀 학습 시작

############################################################
🌱 SEED 1/3 (seed=42)
############################################################

  📂 Fold 1/5


AttributeError: 'list' object has no attribute 'to'